# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'api_keys'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df=pd.read_csv('citydata.csv')
city_df

,Country,City,Lat,Lnt,Temperature,Max_Temp,Humidity,Cloudiness,Wind_Speed
0,id,poso,-1.3959,120.7524,87.33,87.33,61,65,5.59
1,ar,ushuaia,-54.8000,-68.3000,44.60,44.60,70,20,9.22
2,br,montanha,-18.1269,-40.3633,70.65,70.65,96,4,3.20
3,ru,sangar,63.9242,127.4739,-40.67,-40.67,79,99,6.38
4,au,busselton,-33.6500,115.3333,74.46,75.99,42,1,5.99
...,...,...,...,...,...,...,...,...,...
703,br,porto walter,-8.2686,-72.7439,73.71,73.71,95,98,2.53
704,za,plettenberg bay,-34.0527,23.3716,69.08,69.08,92,54,8.57
705,us,moberly,39.4184,-92.4382,39.96,41.00,70,1,10.36
706,et,dembi dolo,8.5333,34.8000,60.51,60.51,41,48,3.44


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = city_df[["Lat", "Lnt"]].astype(float)
humidity=city_df["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#& city_df.Wind_Speed.isin([0, 9])

new_df=city_df.loc[city_df['Max_Temp']<80]
new_df=city_df.loc[city_df['Max_Temp']>80]
new_df=city_df.loc[city_df['Wind_Speed']<10]
new_df=city_df.loc[city_df['Cloudiness']==0]
new_df

,Country,City,Lat,Lnt,Temperature,Max_Temp,Humidity,Cloudiness,Wind_Speed
6,in,dhilwan,31.5169,75.3500,50.07,50.07,47,0,3.13
11,za,saldanha,-33.0117,17.9442,62.60,62.60,77,0,2.30
12,cn,dongsheng,39.8161,109.9776,25.47,25.47,66,0,15.17
28,za,cape town,-33.9258,18.4232,70.56,71.60,73,0,12.66
29,sc,victoria,22.2855,114.1577,59.43,62.01,26,0,4.61
...,...,...,...,...,...,...,...,...,...
661,cl,tome,-36.6167,-72.9500,53.60,53.60,76,0,6.91
664,ci,korhogo,9.4500,-5.6333,70.61,70.61,24,0,1.77
680,in,narasannapeta,18.4167,84.0500,79.09,79.09,56,0,3.36
694,cn,leshan,29.5623,103.7639,48.76,48.76,51,0,2.19


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# find the closest restaurant of each type to coordinates
radius=5000
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type": "lodging","key": g_key,"radius":radius}
#new_df.loc[:,"Hotel_Name"]=""
#new_df.loc[:,"Hotel_Address"]=""
#new_df.loc[:,"Hotel_Lat"]=""
#new_df.loc[:,"Hotel_Lnt"]=""
hotel_df=new_df
# use iterrows to iterate through pandas dataframe

for index, row in new_df.iterrows():

    # get restaurant type from df
    location_lat = str(round(row['Lat'],6))
    location_lnt = str(round(row['Lnt'],6))
    params['location']=location_lat+","+location_lnt
    print(f"Retrieving Results for Index {index}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        print(f"Closest Hotel is {results[0]['name']}.")        
        hotel_df.loc[index,'Hotel name']=results[0]['name']
        hotel_df.loc[index,'Hotel_Address'] = results[0]['vicinity']
        hotel_df.loc[index,'Hotel_Lat']=results[0]['geometry']['location']['lat']
        hotel_df.loc[index,'Hotel_Lnt']=results[0]['geometry']['location']['lng']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Retrieving Results for Index 6.
Closest Hotel is Janj Ghar.
Retrieving Results for Index 11.


C:\Users\Oscar\.conda\envs\PythonData\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Oscar\.conda\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest Hotel is Blue Bay Lodge.
Retrieving Results for Index 12.
Closest Hotel is Crowne Plaza Ordos.
Retrieving Results for Index 28.
Closest Hotel is Radisson Blu Hotel Waterfront, Cape Town.
Retrieving Results for Index 29.
Closest Hotel is Mini Hotel Central.
Retrieving Results for Index 32.
Missing field/result... skipping.
Retrieving Results for Index 34.
Closest Hotel is Bjarnabúð.
Retrieving Results for Index 42.
Closest Hotel is Mövenpick Hotel Qassim.
Retrieving Results for Index 44.
Closest Hotel is Pension Maro'i.
Retrieving Results for Index 51.
Closest Hotel is Hotel Boutique Las Azucenas.
Retrieving Results for Index 57.
Closest Hotel is Case de Passage De la Prefecture.
Retrieving Results for Index 59.
Closest Hotel is Kvitbrakka.
Retrieving Results for Index 63.
Closest Hotel is Hotel Arena Gruesa.
Retrieving Results for Index 69.
Closest Hotel is Gran Hotel Mar del Plata.
Retrieving Results for Index 76.
Closest Hotel is Desert Gardens Hotel - Ayers Rock Resort.
Retr

In [7]:
hotel_df

,Country,City,Lat,Lnt,Temperature,Max_Temp,Humidity,Cloudiness,Wind_Speed,Hotel_name,Hotel_Address,Hotel_Lat,Hotel_Lnt
6,in,dhilwan,31.5169,75.3500,50.07,50.07,47,0,3.13,Janj Ghar,"Dhilwan Nadala Link Road, Dhilwan",31.512476,75.342667
11,za,saldanha,-33.0117,17.9442,62.60,62.60,77,0,2.30,Blue Bay Lodge,"48 Gracillaria Cresent, Blouwater Bay, Saldanha",-32.995413,17.973140
12,cn,dongsheng,39.8161,109.9776,25.47,25.47,66,0,15.17,Crowne Plaza Ordos,China,39.828710,109.961820
28,za,cape town,-33.9258,18.4232,70.56,71.60,73,0,12.66,"Radisson Blu Hotel Waterfront, Cape Town","100 Beach Road, Granger Bay Boulevard, Cape Town",-33.899615,18.412039
29,sc,victoria,22.2855,114.1577,59.43,62.01,26,0,4.61,Mini Hotel Central,"38 號 Ice House Street, Central",22.280489,114.156340
...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,cl,tome,-36.6167,-72.9500,53.60,53.60,76,0,6.91,Dichato habitaciones,"Pablo Ivelic 1350, Dichato",-36.620854,-72.951695
664,ci,korhogo,9.4500,-5.6333,70.61,70.61,24,0,1.77,Hôtel Tchinbota,"B308, Korhogo",9.447663,-5.631399
680,in,narasannapeta,18.4167,84.0500,79.09,79.09,56,0,3.36,G.lingaraju home,"door no :22-44 Gandhi nagar line, 1, Singupura...",18.416960,84.048812
694,cn,leshan,29.5623,103.7639,48.76,48.76,51,0,2.19,Leshan Haiyun International Youth Hostel,"59 Renmin South Road, Shizhong District, Lesha...",29.559977,103.763059


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel_name</dt><dd>{Hotel_name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lnt"]]

In [23]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=5,
    info_box_content=[f"{Hotel}" for Hotel in hotel_info]
)
# Display figure
fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))